# Obtaining the starting point for *yambo*: *QuantumEspresso* self and non self-consistent simulations.

***Prerequisites***: we suppose that you have successfully installed and configured AiiDA, aiida-quantuespresso and that you have stored computer, codes, structures and pseudo in the AiiDA database.

The following part explains how to run the density functional theory (DFT) simulations, using as example
the hexagonal boron nitride (hBN). 
The starting point is a self-consistent field (SCF) calculation of the electronic density, 
and then a calculation of the electronic wavefunctions through a non self-consistent (NSCF) DFT calculation. 
So, the first AiiDA plugin used here is *aiida-quantumespresso*. 
This i required by *yambo* in order to perform GW and BSE simulations. For more details, please have a look at the [AiiDA tutorials](https://aiida-tutorials.readthedocs.io/en/latest/). 

Simulations here are performed by means of the [*QuantumEspresso*](https://www.quantum-espresso.org/) simulation package, in particular using the *pw.x** executable. Pseudpotentials employed will be the [PseudoDojo](http://www.pseudo-dojo.org/) ones.

In [ ]:
from aiida import orm, load_profile
load_profile()

from aiida.plugins import WorkflowFactory
from aiida.orm import QueryBuilder
from aiida.engine import submit

from aiida_quantumespresso.common.types import ElectronicType

import yaml

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

We first load the codes and resources settings, written in the corresponding yaml files:

In [2]:
# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)

In [3]:
codes,resources

({'pwcode_id': 'qe.pw@local_slurm',
  'yambocode_id': 'yambo-5.2.1@local_slurm',
  'yamboprecode_id': 'p2y-5.2.1@local_slurm'},
 {'max_wallclock_seconds': 3600,
  'prepend_text': 'export OMP_NUM_THREADS=1',
  'resources': {'num_cores_per_mpiproc': 1,
   'num_machines': 1,
   'num_mpiprocs_per_machine': 1}})

## (1) Self-consistent field simulation.

This is step is dedicated to the calculation of the ground state density by means of a self-consistent simulation. This is performed via a submission of the `PwBaseWorkChain` of the ***aiida-quantumespresso*** plugin. This workflow provides automatic error handling and automatic input creation (taking advantage of a pre-defined set of protocols for the determination of default parameters). 
We are going to use the ``pk`` of the SCF 
calculation in the next step (NSCF). The ``pk`` is the number that identifies the corresponding node 
in the AiiDA database, and can be accessed via:
```bash
In  [1]: given_node.pk
Out [2]: 1234  #pk of the node here named "given_node".
```

For more details in the ***aiida-quantumespresso*** plugin you can have a look at the corresponding [documentation](https://aiida-quantumespresso.readthedocs.io/en/latest/).

### Loading the structure via QueryBuilder
The first operation we do is to load the structure from our aiida database (we consider you completed the previous notebook on structure and pseudos setting):

In [3]:
qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'hBN/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure = qb.all()[0][0]

Inspect the `loaded_structure` object:

In [ ]:
loaded_structure

The last ingredient needed to run our simulation is the class of the AiiDA WorkChain we are going to submit:

In [ ]:
PwBaseWorkchain = WorkflowFactory('quantumespresso.pw.base') # WorkflowFactory(workchain-entry-point)

You can try also to load the `PwRelaxWorkchain` class, used to perform structure relaxation. The corresponding entry-point is `quantumespresso.pw.relax`. 
Here, we that our input structure is already relaxed.

In the following we create the *builder* instance, which collects all the inputs needed in our `PwBaseWorkchain` to be executed. We will use the `get_builder_from_protocol` methods to pre-populate the builder with default parameters. 

In [4]:
builder = PwBaseWorkchain.get_builder_from_protocol(
    code=orm.load_code(codes["pwcode_id"]),
    structure=loaded_structure,               # <=== this is the structure stored before, in 01_structure_and_pseudos.ipynb
    protocol='fast',
    overrides={},
    electronic_type=ElectronicType.INSULATOR, # <=== disabling the smearing
    
)

NotExistent: no Code found with LABEL<qe.pw@local_slurm>: No result was found

We can inspect the inputs parameters, which will be used to create the *pw.x* input file. The parameters are stored in an AiiDA *Dict*, which can be dumped to a python dictionary using the `get_dict()` method:

In [6]:
builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

As you can notice, some of the *pw.x* input parameters, such as `outdir`, are not set: this will be set automatically by the plugin, you do not have to deal with folders and paths.

### Change the pseudos to Norm-Conserving

We change the pseudo family to be the [PseudoDojo](http://www.pseudo-dojo.org/) one. 
The default family is the [SSSP](https://www.materialscloud.org/discover/sssp/table/efficiency) one, but here we need only Norm-Conserving pseudopotentials. To define custom pseudopotential families have a look at the [previous tutorial](0_1_structure_and_pseudos.ipynb) or [here](https://aiida-pseudo.readthedocs.io/en/latest/howto.html).

In [9]:
family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=loaded_structure) 

## How to override the default inputs

It is possible to modify in two ways the default inputs:

    (1) to use the overrides in the builder creation;
    (2) to define again the input variables after the builder instance generation.

### (1) Overrides

Overrides essentially act during the builder instance creation and, as the name suggests, ovverride the protocols inputs.

In [11]:
# (1):

overrides={
    'pw':{
        'parameters':{
            'SYSTEM': {
            'nosym': False,
            'occupations': 'fixed',
            'ecutwfc': 70.0,
            'ecutrho': 70*4,
            'force_symmorphic':True,
                      },
    },},}

builder = PwBaseWorkchain.get_builder_from_protocol(
    code=orm.load_code(codes["pwcode_id"]),
    structure=loaded_structure,
    protocol='fast',
    overrides=overrides,
    electronic_type=ElectronicType.INSULATOR,    
)

builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic': True},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

### (2) Re-define input parameters

The second way is to overwrite the inputs a posteriori, i.e. after the builder instance is created.

In [12]:
# (2):

builder = PwBaseWorkchain.get_builder_from_protocol(
    code=orm.load_code(codes["pwcode_id"]),
    structure=loaded_structure,
    protocol='fast',
    overrides={},
    electronic_type=ElectronicType.INSULATOR,    
)

family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=loaded_structure) 
builder.pw.pseudos = family.get_pseudos(structure=loaded_structure) 

new_pw_parameters = {
  'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.002,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic':True,},
 'ELECTRONS': {'electron_maxstep': 90, 'mixing_beta': 0.4, 'conv_thr': 8e-10},
}

builder.pw.parameters = orm.Dict(dict=new_pw_parameters)

builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.002,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic': True},
 'ELECTRONS': {'electron_maxstep': 90, 'mixing_beta': 0.4, 'conv_thr': 8e-10}}

Within protocols, the kpoints mesh is automatically choosen, by means of a `kpoints_distance` (inverse of the k-points density in the reciprocal space):

In [13]:
builder.kpoints_distance.value

0.5

We can define our desired k-points mesh, by creating an istance of `KpointsData` and then setting the `builder.kpoints` attribute again:

In [14]:
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([4,4,4])

builder.kpoints = kpoints

### Leaving the calculation folder available for next simulations:

Also, we set the workchain input `clean_workdir` to `False`, so we do not delete the remote folder after the simulation is finished (we need the charge-density to run the nscf step):

In [15]:
builder.clean_workdir = orm.Bool(False)

## Setting the computational resources
 
Then you should set up the options about resources and submission settings. Here is where you can specify options like the partition name of the cluster (`queue_name`), the `account` and more.
These information have to be provided as a python dictionary in the metadata method:

In [20]:
builder.pw.metadata.options = resources

## Run

Here we submit the simulation to the AiiDA daemon via the `submit` function. In this way, even if close the current session (i.e. we shutdown the notebook) AiiDA will continue to take care of monitoring the simulation up to its completion.

In [22]:
run_scf = submit(builder)

We can see that the `run_scf` variable is indeed our `PwBaseWorkchain` just submitted:

In [23]:
print("PwBaseWorkchain instance:", run_scf)
print("PwBaseWorkchain pk:", run_scf.pk)
print("PwBaseWorkchain uuid:", run_scf.uuid)

PwBaseWorkchain instance: uuid: bf8d58e3-37d9-40ad-8708-8c9e67b5d897 (pk: 365) (aiida.workflows:quantumespresso.pw.base)
PwBaseWorkchain pk: 365
PwBaseWorkchain uuid: bf8d58e3-37d9-40ad-8708-8c9e67b5d897


You can monitor the calculation from a shell, executing one or more of the following instructions:

```bash
    verdi process list
    verdi process show <pk of the run_scf>
    verdi process report <pk of the run_scf>
```

It is possible to check if the calculation `is_finished_ok`:

In [24]:
run_scf.is_finished_ok

True

## Outputs inspection

If `run_scf.is_finished_ok`, we can start inspecting the outputs of the calculation.

In [25]:
run_scf.outputs.output_parameters.get_dict()   #use tab to see available outputs

{'lsda': False,
 'energy': -729.59295119727,
 'volume': 34.754548090754,
 'fft_grid': [25, 25, 72],
 'energy_xc': -236.50883655993,
 'wall_time': '      8.60s ',
 'rho_cutoff': 3809.593683084,
 'symmetries': [{'t_rev': '0', 'symmetry_number': 0},
  {'t_rev': '0', 'symmetry_number': 2},
  {'t_rev': '0', 'symmetry_number': 26},
  {'t_rev': '0', 'symmetry_number': 27},
  {'t_rev': '0', 'symmetry_number': 28},
  {'t_rev': '0', 'symmetry_number': 29},
  {'t_rev': '0', 'symmetry_number': 33},
  {'t_rev': '0', 'symmetry_number': 35},
  {'t_rev': '0', 'symmetry_number': 56},
  {'t_rev': '0', 'symmetry_number': 57},
  {'t_rev': '0', 'symmetry_number': 62},
  {'t_rev': '0', 'symmetry_number': 63}],
 'wfc_cutoff': 952.398420771,
 'format_name': 'QEXSD',
 'lkpoint_dir': False,
 'occupations': 'fixed',
 'total_force': 0.0,
 'creator_name': 'pwscf',
 'energy_ewald': -516.99706371224,
 'energy_units': 'eV',
 'fermi_energy': 4.7464793876906,
 'forces_units': 'ev / angstrom',
 'q_real_space': False,
 '

## Run the NSCF step

Using the previous completed SCF calculation, we now run a NSCF calculation as the starting point for the GW calculation. Here we will just use the `pk` of the previous calculation, but you can also put node in an AiiDA group to load it without the need to remember the `run_scf.pk` value, in a similar manner on how we do for the structure.

In [27]:
overrides={
    'pw':{
        'parameters':{
            'CONTROL': {
                'calculation': 'nscf',
                'restart_mode':'restart',},
            'SYSTEM': {
            'nosym': False,
            'occupations': 'fixed',
            'ecutwfc': 70.0,
            'ecutrho': 70*4,
            'nbnd':200,                     # <=== we increase the number of bands.
            'force_symmorphic':True,
                      },
    },},}

builder = PwBaseWorkchain.get_builder_from_protocol(
    
                code=orm.load_code(codes["pwcode_id"]),
                structure=loaded_structure,
                protocol='fast',
                overrides=overrides,
                electronic_type=ElectronicType.INSULATOR,
                pseudo_family="PseudoDojo/0.4/PBE/SR/standard/upf",
    
)

#If you want to change the pseudos:
family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=loaded_structure) 

builder.pw.metadata.options = resources

builder.clean_workdir = orm.Bool(False)    # we need the workdir for yambo pre-processing step (p2y)

### Setting the parent scf calculation as starting point for the nscf.

We set as parent_folder the previous completed scf run. 
This needs to be provided by means of the `builder.pw.parent_folder` attribute.

In [28]:
parent_scf = orm.load_node(run_scf.pk) 
builder.pw.parent_folder = parent_scf.outputs.remote_folder

In [29]:
run_nscf = submit(builder)

In [30]:
print("PwBaseWorkchain instance:", run_nscf)
print("PwBaseWorkchain pk:", run_nscf.pk)
print("PwBaseWorkchain uuid:", run_nscf.uuid)

379

In [31]:
run_nscf.is_finished_ok

True

In [ ]:
run_nscf.outputs.output_parameters.get_dict()   #use tab to see available outputs